In [52]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
pwd

'/content'

In [54]:
# read CSVs
mentor = pd.read_csv(r"/content/Data.csv", index_col=False)
mentor.head(3)

,name,identity,sex,city,city_tier,sector,company,designation,shortlistedforcompany,college,College_tier,College Tier,program,department,ssc,hsc,cgpa,internship,other,postcity,othrahiev,gdduration,threewddcrua,is_active
0,Krati Tiwari,4,F,Indore,II,Technology,Google Inc.,Software Engineer,Pre-placement offer,IIT Bombay,Tier I,Tier I,B.Tech,Computer Science,100,86.2,7.40,G Suite & Tez App teams in Google Inc,Pre-placement offer,Hyderabad,"Brand Ambassadors of Digital India, Desai Seth...",2015-2019,Techie | Cricketer | Drama Enthusiast,Y
1,Avineil Jain,6,M,Vapi,III,Finance,Optiver,Trader,"Blackstone, WorldQuant, JP Morgan, QuadEye Sec...",IIT Bombay,Tier I,Tier I,Dual Degree,Electrical Engineering,100,96.4,9.54,"Schlumberger software services, George Washing...","Blackstone, WorldQuant, JP Morgan, QuadEye Sec...",Amsterdam,TFI LEaRN Scholarship,2014-2019,Golfer | Techie | Scuba Diver,Y
2,Prathu Baronia,7,M,Delhi,I,Technology,OnePlus,R &D Software Developer,"Xilinx, Analog Devices, Mathworks, Nvidia, Qua...",IIT Bombay,Tier I,Tier I,Dual Degree,Electrical Engineering,90.4,90.4,9.00,Embedded System Engineer at Greetude Energy Pv...,"Xilinx, Analog Devices, Mathworks, Nvidia, Qua...",Bangalore,"98.1%ile in NSE Astronomy-2014, 99%ile in NSE ...",2014-2019,Musician | Cricketer | Techie,Y


In [55]:
mentor.index

RangeIndex(start=0, stop=405, step=1)

In [56]:
mentor.shape

mentor.isnull().any()

name                     False
identity                 False
sex                       True
city                     False
city_tier                False
sector                   False
company                  False
designation               True
shortlistedforcompany     True
college                  False
College_tier             False
College Tier              True
program                  False
department                True
ssc                       True
hsc                       True
cgpa                      True
internship                True
other                     True
postcity                  True
othrahiev                 True
gdduration                True
threewddcrua              True
is_active                 True
dtype: bool

In [57]:
mentor['ssc'].value_counts()

95        32
100        8
89         6
10         6
91.2       5
          ..
75.71%     1
85.17      1
93.14      1
79         1
88%        1
Name: ssc, Length: 127, dtype: int64

In [58]:
#Data frame for user
data = pd.read_csv(r"/content/UserInputData.csv",index_col = False)
data.head(1)

,name,identity,sector,company,college,program,department,ssc,hsc,cgpa,internship,other,postcity,othrahiev,gdduration,threewddcrua,is_active
0,xyz,3,Technology,Microsoft,IIT Kharagpur,Dual Degree,Electrical Engineering,77.83,82.5,7.9,J P Morgan Chase,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
data.shape

(1, 17)

In [60]:
df1=pd.DataFrame(mentor, columns=['identity','name','sector','company','program','college','department','ssc','hsc','cgpa','internship'])
df1.iloc[0:3]

,identity,name,sector,company,program,college,department,ssc,hsc,cgpa,internship
0,4,Krati Tiwari,Technology,Google Inc.,B.Tech,IIT Bombay,Computer Science,100,86.2,7.40,G Suite & Tez App teams in Google Inc
1,6,Avineil Jain,Finance,Optiver,Dual Degree,IIT Bombay,Electrical Engineering,100,96.4,9.54,"Schlumberger software services, George Washing..."
2,7,Prathu Baronia,Technology,OnePlus,Dual Degree,IIT Bombay,Electrical Engineering,90.4,90.4,9.00,Embedded System Engineer at Greetude Energy Pv...


In [61]:
df2=pd.DataFrame(data, columns=['identity','name','sector','company','program','college','department','ssc','hsc','cgpa','internship'])
df2.iloc[0:]

,identity,name,sector,company,program,college,department,ssc,hsc,cgpa,internship
0,3,xyz,Technology,Microsoft,Dual Degree,IIT Kharagpur,Electrical Engineering,77.83,82.5,7.9,J P Morgan Chase


In [62]:
df = df2.append(df1,ignore_index = True)
df.head(3)

,identity,name,sector,company,program,college,department,ssc,hsc,cgpa,internship
0,3,xyz,Technology,Microsoft,Dual Degree,IIT Kharagpur,Electrical Engineering,77.83,82.5,7.90,J P Morgan Chase
1,4,Krati Tiwari,Technology,Google Inc.,B.Tech,IIT Bombay,Computer Science,100,86.2,7.40,G Suite & Tez App teams in Google Inc
2,6,Avineil Jain,Finance,Optiver,Dual Degree,IIT Bombay,Electrical Engineering,100,96.4,9.54,"Schlumberger software services, George Washing..."


In [63]:
#create a list of important columns to keep
features = ['sector','company','program','department','college','cgpa','ssc','hsc','internship']
mentor[features].head(3)

,sector,company,program,department,college,cgpa,ssc,hsc,internship
0,Technology,Google Inc.,B.Tech,Computer Science,IIT Bombay,7.40,100,86.2,G Suite & Tez App teams in Google Inc
1,Finance,Optiver,Dual Degree,Electrical Engineering,IIT Bombay,9.54,100,96.4,"Schlumberger software services, George Washing..."
2,Technology,OnePlus,Dual Degree,Electrical Engineering,IIT Bombay,9.00,90.4,90.4,Embedded System Engineer at Greetude Energy Pv...


In [64]:
#clean and process the data
for feature in features:
 df[feature] = df[feature].fillna('')  #fill any missing value with the empty string

In [65]:
#create a function to combine the values of the important columns
def combine_features(row):
    return row['sector']+" "+row['company']+" "+row['program']+" "+row['department']+" "+row['college']+" "+row['internship']

In [66]:
#apply the function to each row in the data set to store the combined strings into a new column called combined_features
df['combined_features'] = df.apply(combine_features, axis = 1)

In [67]:
df['combined_features'].head(10)

0    Technology Microsoft Dual Degree Electrical En...
1    Technology Google Inc. B.Tech Computer Science...
2    Finance Optiver Dual Degree Electrical Enginee...
3    Technology OnePlus Dual Degree Electrical Engi...
4    Media Viacom 18 B.Tech Mechanical Engineering ...
5    Consulting IMS Consulting B.Tech Chemical Engi...
6    Oil and Gas ExxonMobil B.Tech Chemical Enginee...
7    Consulting BCG B.Tech Mechanical  IIT Bombay R...
8    Finance JP Morgan Chase B.Tech Electrical Engi...
9    Analytics Indegene Pvt. Ltd. B.Tech Metallurgy...
Name: combined_features, dtype: object

In [68]:
!pip install normalise

In [69]:
!pip install nltk

In [70]:
import numpy as np
import multiprocessing as mp
import nltk
nltk.download('brown')
nltk.download('names')
import string
import spacy 
import en_core_web_sm
from nltk.tokenize import word_tokenize
from sklearn.base import TransformerMixin, BaseEstimator
from normalise import normalise

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [71]:
nlp = en_core_web_sm.load()


class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self,
                 variety="BrE",
                 user_abbrevs={},
                 n_jobs=1):
        """
        Text preprocessing transformer includes steps:
            1. Text normalization
            2. Punctuation removal
            3. Stop words removal
            4. Lemmatization
        
        variety - format of date (AmE - american type, BrE - british format) 
        user_abbrevs - dict of user abbreviations mappings (from normalise package)
        n_jobs - parallel jobs to run
        """
        self.variety = variety
        self.user_abbrevs = user_abbrevs
        self.n_jobs = n_jobs

    def fit(self, X, y=None):
        return self

    def transform(self, X, *_):
        X_copy = X.copy()

        partitions = 1
        cores = mp.cpu_count()
        if self.n_jobs <= -1:
            partitions = cores
        elif self.n_jobs <= 0:
            return X_copy.apply(self._preprocess_text)
        else:
            partitions = min(self.n_jobs, cores)

        data_split = np.array_split(X_copy, partitions)
        pool = mp.Pool(cores)
        data = pd.concat(pool.map(self._preprocess_part, data_split))
        pool.close()
        pool.join()

        return data

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        normalized_text = self._normalize(text)
        doc = nlp(normalized_text)
        removed_punct = self._remove_punct(doc)
        removed_stop_words = self._remove_stop_words(removed_punct)
        return self._lemmatize(removed_stop_words)

    def _normalize(self, text):
        # some issues in normalise package
        try:
            return ' '.join(normalise(text, variety=self.variety, user_abbrevs=self.user_abbrevs, verbose=False))
        except:
            return text

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if not t.is_stop]

    def _lemmatize(self, doc):
        return ' '.join([t.lemma_ for t in doc])

In [72]:
%%time
text = TextPreprocessor(n_jobs=-1).transform(df['combined_features'])
#df['combined_features'].head(10)

CPU times: user 12.3 ms, sys: 31 ms, total: 43.3 ms
Wall time: 3.27 s


In [73]:
text = TextPreprocessor(n_jobs=-1).transform(df['combined_features'])

In [74]:
df['combined_features'].head(10)

0    Technology Microsoft Dual Degree Electrical En...
1    Technology Google Inc. B.Tech Computer Science...
2    Finance Optiver Dual Degree Electrical Enginee...
3    Technology OnePlus Dual Degree Electrical Engi...
4    Media Viacom 18 B.Tech Mechanical Engineering ...
5    Consulting IMS Consulting B.Tech Chemical Engi...
6    Oil and Gas ExxonMobil B.Tech Chemical Enginee...
7    Consulting BCG B.Tech Mechanical  IIT Bombay R...
8    Finance JP Morgan Chase B.Tech Electrical Engi...
9    Analytics Indegene Pvt. Ltd. B.Tech Metallurgy...
Name: combined_features, dtype: object

In [47]:
#convert a collection of text to a matrix of token counts
count_matrix = CountVectorizer().fit_transform(df['combined_features'])

In [48]:
#get the cosine similarity matrix from the count matrix
cosine_sim = cosine_similarity(count_matrix)
print(cosine_sim)

[[1.         0.14509525 0.40824829 ... 0.         0.         0.        ]
 [0.14509525 1.         0.11846978 ... 0.         0.         0.        ]
 [0.40824829 0.11846978 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         1.         1.        ]
 [0.         0.         0.         ... 1.         1.         1.        ]
 [0.         0.         0.         ... 1.         1.         1.        ]]


In [ ]:
#get the number of rows and columns in cosine_sim
cosine_sim.shape

(406, 406)

In [ ]:
#helper function to get the name from the id
def name_identity(identity):
    return df[df.identity == identity]['name'].values[0]

In [ ]:
#helper function to get the id from the name
def identity_name(name):
    return df[df.name == name]['identity'].values[0] 

In [ ]:
#the name of the user
user = df.loc[0,"name"]
user

'xyz'

In [ ]:
#Find that name id
name_id = identity_name(user)

In [ ]:
id_name = name_identity(2302)

In [ ]:
name_id

3

In [ ]:
id_name

'Anant Singh'

In [ ]:
#enumerate through all the similarity scores of the 'user' to make a tuple of id and similarity index
similar_mentors = list( enumerate(cosine_sim[name_id]) )

In [ ]:
similar_mentors

[(0, 0.47434164902525694),
 (1, 0.1720618004029213),
 (2, 0.3872983346207416),
 (3, 1.0),
 (4, 0.3818813079129867),
 (5, 0.26064301757134345),
 (6, 0.1767766952966369),
 (7, 0.14433756729740646),
 (8, 0.2886751345948129),
 (9, 0.2672612419124244),
 (10, 0.2004459314343183),
 (11, 0.2785430072655778),
 (12, 0.4472135954999579),
 (13, 0.1767766952966369),
 (14, 0.10660035817780521),
 (15, 0.10425720702853739),
 (16, 0.1720618004029213),
 (17, 0.1336306209562122),
 (18, 0.2165063509461097),
 (19, 0.18190171877724973),
 (20, 0.07537783614444091),
 (21, 0.0),
 (22, 0.15075567228888181),
 (23, 0.08333333333333333),
 (24, 0.10206207261596577),
 (25, 0.0),
 (26, 0.1720618004029213),
 (27, 0.25),
 (28, 0.2165063509461097),
 (29, 0.4082482904638631),
 (30, 0.05892556509887897),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 0.05590169943749474),
 (35, 0.07537783614444091),
 (36, 0.2165063509461097),
 (37, 0.16666666666666666),
 (38, 0.2946278254943948),
 (39, 0.2165063509461097),
 (40, 0.158113883008

In [ ]:
#sort the list of similar mentors in the order of similarity scores in descending order
sorted_similar_mentors = sorted(similar_mentors, key = lambda x:x[1], reverse = True)[2:]

In [ ]:
#print
sorted_similar_mentors

[(12, 0.4472135954999579),
 (29, 0.4082482904638631),
 (198, 0.39528470752104744),
 (2, 0.3872983346207416),
 (4, 0.3818813079129867),
 (43, 0.35355339059327373),
 (171, 0.33407655239053047),
 (258, 0.3333333333333333),
 (222, 0.3287979746107146),
 (187, 0.31622776601683794),
 (67, 0.30316953129541624),
 (38, 0.2946278254943948),
 (8, 0.2886751345948129),
 (42, 0.2886751345948129),
 (311, 0.28347335475692037),
 (133, 0.2795084971874737),
 (11, 0.2785430072655778),
 (204, 0.2773500981126146),
 (9, 0.2672612419124244),
 (5, 0.26064301757134345),
 (190, 0.2551551815399144),
 (213, 0.2551551815399144),
 (27, 0.25),
 (49, 0.25),
 (168, 0.25),
 (65, 0.23717082451262844),
 (192, 0.23717082451262844),
 (227, 0.23570226039551587),
 (51, 0.22613350843332272),
 (87, 0.22613350843332272),
 (141, 0.22613350843332272),
 (260, 0.22613350843332272),
 (357, 0.22613350843332272),
 (256, 0.22360679774997896),
 (335, 0.22360679774997896),
 (18, 0.2165063509461097),
 (28, 0.2165063509461097),
 (36, 0.21650

In [ ]:
i = 0
print ('The top 5 similar mentor to '+user+' are:')
for element in sorted_similar_mentors:
    print(element[0])
    i = i + 1
    if i >= 5:
        break


The top 5 similar mentor to xyz are:
12
29
198
2
4


In [ ]:
identities =[x[0] for x in sorted_similar_mentors[:5]]
identities

[12, 29, 198, 2, 4]

In [ ]:
x=identities

In [ ]:
result={"Name":[],"Identity":[]}
for i in x :
    result['Name'].append(df.loc[i,"name"])
    result['Identity'].append(df.loc[i,"identity"])

In [ ]:
import pandas as pd
result=pd.DataFrame(result)

In [ ]:
result

,Name,Identity
0,Sawan Soni,18
1,Rashish Rajendra Shingi,104
2,Biswaranjan Naik,1514
3,Avineil Jain,6
4,Shubham Shah,8
